# Task 2 DF

In [0]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from operator import add
import operator
from functools import reduce
from datetime import datetime
import pyspark.sql.types as types
from pyspark.sql.types import DateType, LongType
from pyspark.sql.functions import col, avg, min, max, round, count as _count, broadcast

spark = SparkSession.builder.appName("task2").getOrCreate()
stagemetrics = StageMetrics(spark)

dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
year = 2000

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def compute_delay(time_departed, time_scheduled):
      # Assume there are no delays longer than 12 hours
      # There are no flights early by more than 12 hours
          
      # Fill the rest of the 24 hour time (e.g 15 -> 0015)
      time_departed = str(time_departed).zfill(4)
      time_scheduled = str(time_scheduled).zfill(4)
      
      # Check for 2400 if so convert to 0000
      if time_departed == "2400":
        time_departed = "0000"
      elif time_scheduled == "2400":
        time_scheduled = "0000"
      
      # Convert hours to minutes
      td_hours_to_mins = int(time_departed[:-2]) * 60
      # Add the remaining minutes to get total minutes
      time_departed_mins = td_hours_to_mins + int(time_departed[-2:])
      # Convert hours to minutes
      ts_hours_to_mins = int(time_scheduled[:-2]) * 60
      # Add the remaining minutes to get total minutes
      time_scheduled_mins = ts_hours_to_mins + int(time_scheduled[-2:])
      
      # Calculate Delay
      delay = time_departed_mins - time_scheduled_mins
      
      # Check if there is a delay longer than 12 hours
      if delay/60 > 12:
        return delay - 1440
      # Check there are no flights early by more than 12 hours
      elif delay/60 < -12:
        return delay + 1440
      else:
        return delay
    
def task_2_df(spark_session, flights_path, airlines_path, year): 
    flights_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(flights_path)
    airlines_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(airlines_path)
    # Clean
    flights = clean_column_names(flights_df)
    airlines = clean_column_names(airlines_df)
    
    # Project only columns that are required
    flights = flights.select("carrier_code", "flight_date", "actual_departure_time", "scheduled_depature_time")
    
    # Filter US country only 
    airlines_df = airlines.filter(F.col("country") == "United States")
    
    # Change dates from string to DateType in Flights
    flight_df = flights.withColumn('flight_date', flights['flight_date'].cast(DateType()))
    
    # Filter flights only in the specified year
    flight_df = flight_df.filter(F.col("flight_date").between(str(year)+'-01-01',str(year)+'-12-31'))
    
    # Filter out NULL
    flight_df = flight_df.filter(F.col("actual_departure_time").isNotNull())
    flight_df = flight_df.filter(F.col("scheduled_depature_time").isNotNull())
    
    # Set up compute delay function and specifiy return type
    compute_delay_udf = F.udf(compute_delay, types.IntegerType())
    
    # Compute the delay
    flight_df = flight_df.withColumn("delay", compute_delay_udf(F.col("actual_departure_time"),F.col("scheduled_depature_time"))) 
    
    # Filter for delayed flights only
    flight_df = flight_df.filter(F.col("delay") > 0)
    
    # Join the tables
    df = flight_df.join(broadcast(airlines_df), flight_df.carrier_code == airlines_df.carrier_code)
    
    # Output dataframe with aggregates
    output_df = df.groupBy("name").agg(_count("name").alias("num_delays"), round(avg("delay"),2).alias("average_delay"), min("delay").alias("min_delay"), max("delay").alias("max_delay")).orderBy("name")
    
    # Printing with desired format
    for name, num_delays, average_delay, min_delay, max_delay in output_df.collect():
      print("{} \t {} \t {} \t {} \t {}".format(name, num_delays, average_delay, min_delay, max_delay))

#     output_df.write.format("com.databricks.spark.csv").option("sep", "\t").option("header", "true").csv("/Filestore/data/dataF.csv")

In [0]:
stagemetrics.begin()

task_2_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc. 66 25.18 1 119
American Airlines Inc. 252 30.97 1 379
Continental Air Lines Inc. 127 21.87 1 216
Delta Air Lines Inc. 389 23.88 1 336
Northwest Airlines Inc. 144 33.78 1 237
Southwest Airlines Co. 358 30.04 2 213
US Airways 237 28.46 1 496
United Airlines 376 33.13 1 397

In [0]:
stagemetrics.begin()

task_2_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc. 595 32.06 1 324
American Airlines Inc. 2740 30.07 1 720
Continental Air Lines Inc. 1258 28.09 1 485
Delta Air Lines Inc. 3711 21.57 1 503
Northwest Airlines Inc. 1616 30.51 1 520
Southwest Airlines Co. 3635 27.67 1 350
US Airways 2683 28.48 1 429
United Airlines 3653 35.05 1 690

In [0]:
stagemetrics.begin()

task_2_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc. 6028 30.49 1 588
American Airlines Inc. 27231 30.16 1 720
Continental Air Lines Inc. 12880 29.31 1 665
Delta Air Lines Inc. 38169 22.31 1 606
Northwest Airlines Inc. 15867 29.78 1 706
Southwest Airlines Co. 36874 28.02 1 465
US Airways 26197 27.95 1 618
United Airlines 36834 34.28 1 718

In [0]:
stagemetrics.begin()

task_2_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc. 47835 30.48 1 696
American Airlines Inc. 218659 30.18 1 720
Continental Air Lines Inc. 101973 28.48 1 720
Delta Air Lines Inc. 303248 21.87 1 700
Northwest Airlines Inc. 126611 29.64 1 717
Southwest Airlines Co. 293942 28.12 1 635
US Airways 208389 28.07 1 714
United Airlines 295520 34.35 1 720